# 동영상 Split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import os

In [ ]:
files = os.listdir('/content/drive/MyDrive/미아 데이터_원본 동영상')
files_names = [os.path.splitext(file)[0] for file in files]
len(files_names)

In [ ]:
def save_frames(video_path, save_dir, frame_skip=10):
    # 비디오 파일 열기
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.", f"{video_path}")
        return


    frame_count = 0
    while True:
        # 프레임 읽기
        ret, frame = cap.read()
        if not ret:
            break


        filename_with_ext = os.path.basename(video_path)  # 파일명과 확장자를 추출
        filename_split, _ = os.path.splitext(filename_with_ext)

        # 일정 간격으로 프레임 저장
        if frame_count % frame_skip == 0:
            frame_path = os.path.join(save_dir, "{}_frame_{}.jpg".format(filename_split, frame_count))
            cv2.imwrite(frame_path, frame)
            print(f'{frame_path} 저장 완료')

        frame_count += 1

    # 비디오 파일 닫기
    cap.release()

In [ ]:
# 사용
for file in files_names:
  video_path = f"/content/drive/MyDrive/미아 데이터_원본 동영상/{file}.mp4"
  save_dir = "/content/drive/MyDrive/DATA_LOST_SPLIT_4"
  save_frames(video_path, save_dir)

# Segment Anything Masking(이미지)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
HOME = '/content'
print(HOME)

In [ ]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
from IPython.display import display, Image
import torch
import matplotlib.pyplot as plt

In [ ]:
import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

In [ ]:
CHECKPOINT_PATH = os.path.join("/content/drive/MyDrive/SAM-FineTuning/sam_vit_h_4b8939.pth")
# print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

files = os.listdir('/content/drive/MyDrive/DATA_LOST_SPLIT_4') # save_dir
files_names = [os.path.splitext(file)[0] for file in files]
print(len(files_names), files_names[:10])

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [ ]:
## SAM 적용
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
mask_predictor = SamPredictor(sam)
error = []
model = YOLO(f'{HOME}/yolov8n.pt')

for file, i in zip(files_names, range(len(files_names))):
  try:
    print('\n', i, '번째')

    results = model.predict(source = f"/content/drive/MyDrive/DATA_LOST_SPLIT_4/{file}.jpg", conf=0.40, classes=[0,1]) # conf = 0.4
    image_bgr = cv2.imread(f"/content/drive/MyDrive/DATA_LOST_SPLIT_4/{file}.jpg", cv2.IMREAD_COLOR)

    for box in results[0].boxes.xyxy:
      image_bgr_bbox = cv2.rectangle(image_bgr, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)


    print(f"{file} YOLO 완료")

    image_bgr = cv2.imread(f"/content/drive/MyDrive/DATA_LOST_SPLIT_4/{file}.jpg", cv2.IMREAD_COLOR)

    transformed_boxes = mask_predictor.transform.apply_boxes_torch(results[0].boxes.xyxy, image_bgr.shape[:2])

    mask_predictor.set_image(image_bgr)

    masks, scores, logits = mask_predictor.predict_torch(
        boxes = transformed_boxes,
        multimask_output=False,
        point_coords=None,
        point_labels=None
    )
    masks = np.array(masks.cpu())

    print(f"{file} SAM 완료")

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    image_rgb_ = Image.fromarray(np.array(image_rgb))

    final_mask = None
    for i in range(len(masks) - 1):
      if final_mask is None:
        final_mask = np.bitwise_or(masks[i][0], masks[i+1][0])
      else:
        final_mask = np.bitwise_or(final_mask, masks[i+1][0])

    final_mask_int = final_mask.astype('int32')
    final_mask_int_ = Image.fromarray(np.array((final_mask_int*255).astype('uint8')))


    image_bgr_ = Image.fromarray(image_bgr)

    cv2.imwrite(f'/content/drive/MyDrive/Data_lost_YOLO/{file}.png', image_bgr_bbox) # 저장 경로
    image_rgb_.save(f'/content/drive/MyDrive/Data_lost_image/{file}.png', 'PNG') # 저장 경로
    final_mask_int_.save(f'/content/drive/MyDrive/Data_lost_label/{file}.png', 'PNG') # 저장 경로

    print(f"{file} 저장 완료 \n")
    print('-------------------------------------------------------------')

  except:
    print(i, '번째', f'{file}에러')
    error.append(file)